# Best practice for plotting batch-runs

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import bokeh
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cellpy import cellreader
from cellpy.utils import ica
from cellpy.exceptions import NullData
import holoviews as hv
import altair as alt
%matplotlib inline

### Setting file names and loading data

In [ ]:
my_data = cellreader.CellpyData()
filename = "../testdata/hdf5/20160805_test001_45_cc.h5"
assert os.path.isfile(filename)
my_data.load(filename)
my_data.set_mass(0.1)

## Creating the ica data

In [ ]:
df_ica_charge, df_ica_discharge = ica._dqdv_split_frames(my_data, tidy=False)
tidy_ica_charge, tidy_ica_discharge = ica.dqdv_frames(my_data, split=True)

In [ ]:
df_ica_charge.head()

In [ ]:
tidy_ica_charge.head()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2)

df_ica_charge.plot(x=("voltage", "v"), ax=ax1)
df_ica_discharge.plot(x=("voltage", "v"), ax=ax2, legend=None)
ax1.set_ylabel("dq")
ax1.legend(loc='center left', bbox_to_anchor=(1, 0))

### Trying Bokeh

The main benifit with using Bokeh: interactive html

In [ ]:
from bokeh.io import output_notebook
output_notebook()

In [ ]:
# trick to flatten the multiindex (not sure if HoloViews takes multiindex)
# ica_df.columns = ['_'.join(col).strip() for col in ica_df.columns.values]

In [ ]:
from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral11, Paired, Viridis256
from bokeh.models.glyphs import MultiLine, Line
from bokeh.io import show
import itertools
from bokeh.transform import linear_cmap

In [ ]:
# source = ColumnDataSource(ica_df) # bokeh automatically flattens multiindexed columns using "_".join(k) (i.e. must be str)
# source.data

In [ ]:
source = ColumnDataSource(tidy_ica_charge)

In [ ]:
xrange = DataRange1d()
yrange = DataRange1d()
plot = figure(x_range=xrange, y_range=yrange, plot_width=800, plot_height=300)
plot.line(source=source, x="voltage", y="dq", legend="cycle") #, line_color=linear_cmap('cycle', "Viridis256", 0, 255))
show(plot)
# How can I set a color palette? mappers?

### Plotting with HoloViews

In [ ]:
hv.extension('bokeh')

In [ ]:
curve1 = hv.Curve(tidy_ica_charge, kdims=['voltage', "dq"], vdims=['cycle'], label="charge")
curve2 = hv.Curve(tidy_ica_discharge, kdims=["voltage", "dq"], vdims="cycle", label="discharge")

In [ ]:
(curve1 * curve2).opts(
    width=800, height=900, legend_position='bottom_right', xlim=(0.05, 1.0),
)

In [ ]:
%%opts Curve () NdOverlay [legend_position='right']
curve3 = (hv.Curve(tidy_ica_charge, kdims=['voltage'], vdims=['dq', 'cycle'], label="Charge")
          .groupby("cycle")
          .opts(
              style={"Curve": dict(color=hv.Cycle("RdBu"))}
          )
          .overlay()
          .opts(
              width=800,
          )
         )
curve3

In [ ]:
s = ica.dqdv_frames(my_data)

In [ ]:
s

In [ ]:
type(s)

In [ ]:
plt.plot(s.voltage, s.dq)